In [1]:
import pickle
import math
import numpy as np
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score, precision_score
import matplotlib.pylab as plt
#from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, cross_val_predict, ShuffleSplit
import tsfresh as tf
%matplotlib inline


C:\Users\AOX\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [24]:
df=pd.read_csv('data.csv',header=None)
dfc=df.values

In [31]:
df.head(20)

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,3,0,0,1,1,654.0,0.65,0.64,-1.12,-0.05,-52.19,74.92,-254.36
1,3,0,0,1,2,892.0,0.66,0.57,-0.77,0.22,-16.69,10.38,-281.18
2,3,0,0,1,3,281.0,0.63,0.42,-0.92,0.11,-46.56,93.08,-280.71
3,3,0,0,1,4,358.0,0.61,0.24,0.84,-0.99,317.01,18.46,236.56
4,3,0,0,1,5,472.0,0.65,-0.48,0.23,-0.69,279.54,103.47,-18.27
5,3,0,0,1,6,7.0,0.33,-0.57,0.03,-0.19,-14.57,-107.82,241.41
6,3,1,0,1,1,664.0,0.98,0.03,-1.84,0.12,-48.55,97.91,-403.20
7,3,1,0,1,2,893.0,1.00,-0.07,-1.25,0.32,1.60,5.92,-437.87
8,3,1,0,1,3,269.0,0.93,-0.44,-1.22,0.29,-55.90,108.08,-440.27
9,3,1,0,1,4,429.0,0.86,-0.20,1.49,-0.85,472.85,36.07,293.73


In [154]:
for i in range(0,len(dfc)):
    dfc[i][10]=dfc[i][10]*math.pi/180
    dfc[i][11]=dfc[i][11]*math.pi/180
    dfc[i][12]=dfc[i][12]*math.pi/180
dfc

array([[  3.00000000e+00,   0.00000000e+00,   0.00000000e+00, ...,
         -2.77472150e-04,   3.98317944e-04,  -1.35232451e-03],
       [  3.00000000e+00,   0.00000000e+00,   0.00000000e+00, ...,
         -8.87336690e-05,   5.51860686e-05,  -1.49491510e-03],
       [  3.00000000e+00,   0.00000000e+00,   0.00000000e+00, ...,
         -2.47539822e-04,   4.94866981e-04,  -1.49241631e-03],
       ..., 
       [  2.99000000e+02,   5.90000000e+01,   0.00000000e+00, ...,
          3.93426693e-06,  -4.23093192e-04,   2.89221785e-04],
       [  2.99000000e+02,   5.90000000e+01,   0.00000000e+00, ...,
         -4.32290871e-04,   3.32817716e-04,  -1.86558685e-04],
       [  2.99000000e+02,   5.90000000e+01,   0.00000000e+00, ...,
         -5.75732116e-04,   3.11976734e-04,  -2.09845292e-04]])

In [176]:
dfa=np.zeros((17820,47))
classes=np.zeros(297)
for i in range(17820):
    for k in range(5):
        for m in range(8):
            dfa[i][k*8+m]=dfc[i*6+k][5+m]
    for m in range(7):
        dfa[i][40+m]=dfc[i*6+5][6+m]
    #dfa[i][47]=dfc[i*6][1]
    if(i%60==0):
        n=i/60
        #dfa[i][48]=dfc[i*6][2]*8+dfc[i*6][3]
        classes[int(n)]=dfc[i*6][2]*8+dfc[i*6][3]
classes

array([  1.,   3.,  15.,  42.,  34.,  34.,  26.,  26.,   9.,  12.,  47.,
        11.,  27.,  31.,  31.,  14.,  43.,   4.,  30.,   4.,  42.,  39.,
        32.,  15.,  44.,   9.,  36.,  43.,  40.,  34.,   5.,  24.,  28.,
        12.,  13.,  29.,   1.,  13.,   8.,  10.,  24.,   3.,  28.,  16.,
        45.,  41.,  47.,  28.,  33.,  23.,  16.,  13.,   7.,  21.,   7.,
        39.,  40.,  15.,  37.,   1.,  38.,  28.,  28.,  12.,  34.,  25.,
        28.,  32.,  45.,  14.,  12.,  37.,  20.,  43.,  35.,  39.,  29.,
         0.,  29.,  33.,  21.,  16.,   8.,  31.,   0.,  34.,  12.,   8.,
         9.,  15.,  20.,  10.,  36.,  44.,   9.,  34.,   4.,  29.,  29.,
        33.,  34.,  47.,  16.,  36.,  36.,   7.,  15.,  38.,  47.,  41.,
         4.,  37.,  19.,  33.,   0.,  27.,  19.,  44.,  21.,  26.,   1.,
         7.,  20.,  33.,  38.,  20.,  14.,  23.,  24.,   4.,  15.,  32.,
         8.,  28.,  17.,  31.,  41.,  12.,  15.,  23.,  10.,  43.,  21.,
        23.,  14.,  18.,  27.,  13.,  23.,  23.,  4

In [167]:
dfa[0]

array([  6.54000000e+02,   6.50000000e-01,   6.40000000e-01,
        -1.12000000e+00,  -5.00000000e-02,  -2.77472150e-04,
         3.98317944e-04,  -1.35232451e-03,   8.92000000e+02,
         6.60000000e-01,   5.70000000e-01,  -7.70000000e-01,
         2.20000000e-01,  -8.87336690e-05,   5.51860686e-05,
        -1.49491510e-03,   2.81000000e+02,   6.30000000e-01,
         4.20000000e-01,  -9.20000000e-01,   1.10000000e-01,
        -2.47539822e-04,   4.94866981e-04,  -1.49241631e-03,
         3.58000000e+02,   6.10000000e-01,   2.40000000e-01,
         8.40000000e-01,  -9.90000000e-01,   1.68540805e-03,
         9.81440102e-05,   1.25768944e-03,   4.72000000e+02,
         6.50000000e-01,  -4.80000000e-01,   2.30000000e-01,
        -6.90000000e-01,   1.48619592e-03,   5.50106215e-04,
        -9.71338606e-05,   3.30000000e-01,  -5.70000000e-01,
         3.00000000e-02,  -1.90000000e-01,  -7.74625259e-05,
        -5.73233325e-04,   1.28347484e-03])

In [172]:
df2=pd.DataFrame(dfa)
df2
df22=df2.values
df2.head(5)

,0,1,2,3,4,5,6,7,8,9,...,37,38,39,40,41,42,43,44,45,46
0,654.0,0.65,0.64,-1.12,-0.05,-0.000277,0.000398,-0.001352,892.0,0.66,...,0.001486,0.000550,-0.000097,0.33,-0.57,0.03,-0.19,-0.000077,-0.000573,0.001283
1,664.0,0.98,0.03,-1.84,0.12,-0.000258,0.000521,-0.002144,893.0,1.00,...,0.002132,0.000851,-0.000306,0.67,-0.67,-0.40,-0.41,-0.000126,-0.000750,0.001974
2,664.0,0.99,-1.29,-2.10,0.31,0.000085,0.000319,-0.002325,888.0,1.00,...,0.001961,0.000956,-0.000651,1.00,-0.51,-1.22,-0.62,-0.000238,-0.000578,0.002064
3,665.0,1.00,-1.77,-2.03,0.42,0.000458,-0.000092,-0.002197,892.0,1.00,...,0.001438,0.001191,-0.001043,1.00,-0.39,-1.44,-0.53,-0.000753,-0.000216,0.001948
4,634.0,0.98,-1.65,-1.70,0.04,0.000615,-0.000329,-0.001920,263.0,0.99,...,0.001039,0.001345,-0.001158,1.00,-0.46,-1.69,-0.41,-0.001054,-0.000032,0.001705


In [ ]:
df1=df[df.columns[0:18]]
df1
numsplits=int(len(df1)/20)
numsplits/6
j=0
count=0
for i in range(0,len(df1)):
    df1.iloc[i][17]=int(j)
    count=count+1
    if count==20:
        j=j+1
        count=0
        if j==6:
            j=0

df1
dff=pd.DataFrame(np.zeros((numsplits,120)))
j=0
gesture=0
for i in range(0, numsplits):
    for coli in range(0,17):
        for k in range(0,7):
            dff.iloc[i][(coli*7)+k] = df1.iloc[(i*20):(i*20)+20][coli].describe()[[k+1]]
    dff.iloc[i][119]=gesture
    gesture=gesture+1
    if gesture==6:
        gesture=0